In [ ]:
!pip install fsspec==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets peft accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00


In [ ]:
import torch
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Specify the path where the dataset was saved
saved_path = "/content/drive/MyDrive/Creativity/Data/processed_euclaiseWritingPrompts_dataset_Starling-tokenizer"

# Load the dataset from the saved path
tokenized_dataset = load_from_disk(saved_path)

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Load the tokenizer for your chosen model
model_name = "berkeley-nest/Starling-LM-7B-alpha"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Prepare LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,                # Low-rank adaptation parameter
    lora_alpha=32,           # Scaling factor
    lora_dropout=0.1,          # Dropout to prevent overfitting
    target_modules=[
        f"layers.{i}.self_attn.q_proj" for i in range(20, 32)
    ] + [
        f"layers.{i}.self_attn.k_proj" for i in range(20, 32)
    ] + [
        f"layers.{i}.self_attn.v_proj" for i in range(20, 32)
    ],
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.5 MB/s eta 0:00:00


In [ ]:
training_args = TrainingArguments(
    output_dir="./Starling-7b-finetuned-WritingPrompts",
    run_name="Starling-7b-run-WritingPrompts",
    eval_strategy="no",
    eval_steps=500,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_ratio=0.01,
    lr_scheduler_type="constant",
    save_strategy="steps",
    optim="paged_adamw_8bit",
    max_grad_norm=1.0,
    save_steps=500,
    save_total_limit=5,
    logging_dir="./logs-WritingPrompts",
    fp16=True,
    # remove_unused_columns=False,
    dataloader_num_workers=2
)

tokenized_train = tokenized_dataset["train"]

from transformers import DataCollatorForSeq2Seq
# Define a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator
)

# Fine-tune the model with LoRA
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 18217291136 (18217291136-nanyang-technological-university-singapore). Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
# Define save path
model_save_path = "/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_narrativeqa_long"

# Save the fine-tuned model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_WritingPrompts(reddit_story)/tokenizer_config.json',
 '/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_WritingPrompts(reddit_story)/special_tokens_map.json',
 '/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_WritingPrompts(reddit_story)/tokenizer.model',
 '/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_WritingPrompts(reddit_story)/added_tokens.json',
 '/content/drive/MyDrive/Creativity/Starling_7b_fine_tuned_model_WritingPrompts(reddit_story)/tokenizer.json')